In [1]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

In [2]:
# Display settings
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")

In [3]:
# Load datasets
traffic_df = pd.read_csv('data/Traffic.csv')

In [4]:
traffic_df['Traffic Situation'].value_counts()

Traffic Situation
normal    1669
heavy      682
high       321
low        304
Name: count, dtype: int64

In [5]:
traffic_df.head()

,Time,Date,Day of the week,CarCount,BikeCount,BusCount,TruckCount,Total,Traffic Situation
0,12:00:00 AM,10,Tuesday,31,0,4,4,39,low
1,12:15:00 AM,10,Tuesday,49,0,3,3,55,low
2,12:30:00 AM,10,Tuesday,46,0,3,6,55,low
3,12:45:00 AM,10,Tuesday,51,0,2,5,58,low
4,1:00:00 AM,10,Tuesday,57,6,15,16,94,normal


In [6]:
# Identify and remove outliers
def remove_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

vehicle_counts = ['CarCount', 'BikeCount', 'BusCount', 'TruckCount']
combined_df = remove_outliers(traffic_df, vehicle_counts)

In [7]:
# Check for missing values and duplicates
print("Missing values in each column:")
print(combined_df.isnull().sum())

print(f"Number of duplicate rows: {combined_df.duplicated().sum()}")

Missing values in each column:
Time                 0
Date                 0
Day of the week      0
CarCount             0
BikeCount            0
BusCount             0
TruckCount           0
Total                0
Traffic Situation    0
dtype: int64
Number of duplicate rows: 0


In [8]:
# Prepare the features and target
X = combined_df.drop(columns=['Traffic Situation','BikeCount'])
y = combined_df['Traffic Situation']

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Preprocessing pipeline for numeric and categorical features
numeric_features = ['CarCount', 'BusCount', 'TruckCount', 'Total']
categorical_features = []

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [11]:
# Define the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Train the model
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['CarCount', 'BusCount',
                                                   'TruckCount', 'Total']),
                                                 ('cat', OneHotEncoder(),
                                                  [])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [12]:
# Make predictions
y_pred = model.predict(X_test)

In [13]:
# Evaluate the model
print("Model Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Model Accuracy: 0.9982758620689656
Classification Report:
              precision    recall  f1-score   support

       heavy       1.00      1.00      1.00       118
        high       0.98      1.00      0.99        61
         low       1.00      1.00      1.00        61
      normal       1.00      1.00      1.00       340

    accuracy                           1.00       580
   macro avg       1.00      1.00      1.00       580
weighted avg       1.00      1.00      1.00       580



In [14]:
# Check some predictions 
input_features = X_test.iloc[9].to_dict()
input_features_df = pd.DataFrame([input_features])

predicted_output = model.predict(input_features_df)
print(f"Input features: {input_features}")
print(f"Predicted output for input features: {predicted_output}")

Input features: {'Time': '8:15:00 AM', 'Date': 31, 'Day of the week': 'Tuesday', 'CarCount': 110, 'BusCount': 35, 'TruckCount': 0, 'Total': 179}
Predicted output for input features: ['heavy']


In [15]:
joblib.dump(model, 'model.joblib')

['model.joblib']